In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
# ==========================================================================================================================================================================================

In [17]:
# 1. Device configuration
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# ==========================================================================================================================================================================================

Using device: cpu


In [18]:
# 2. Hyperparameters
# ----------------------------
input_size = 28 * 28   # MNIST images are 28x28
hidden_size = 128
num_classes = 10
num_epochs = 5
batch_size = 64
learning_rate = 0.001

# ==========================================================================================================================================================================================

In [19]:
# 3. Data loading & preprocessing
# ----------------------------
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize with mean & std
])

train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# ==========================================================================================================================================================================================

In [20]:
# 4. Neural Network Model
# ----------------------------
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.fc2 = nn.Linear(hidden_size, num_classes) # Output layer

    def forward(self, x):
        x = x.view(-1, input_size)  # Flatten image
        x = F.relu(self.fc1(x))     # Activation
        x = self.fc2(x)             # Output logits
        return x

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# ==========================================================================================================================================================================================

In [21]:
# 5. Loss and optimizer
# ----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ==========================================================================================================================================================================================

In [22]:
# 6. Training loop
# ----------------------------
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# ==========================================================================================================================================================================================

Epoch [1/5], Loss: 0.2542
Epoch [2/5], Loss: 0.1113
Epoch [3/5], Loss: 0.0771
Epoch [4/5], Loss: 0.0580
Epoch [5/5], Loss: 0.0454


In [23]:
# 7. Evaluation
# ----------------------------
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

# ==========================================================================================================================================================================================

Test Accuracy: 97.64%


In [24]:
# 8. Save the model
# ----------------------------
torch.save(model.state_dict(), "mnist_ffnn.pth")
print("Model saved as mnist_ffnn.pth")
# ==========================================================================================================================================================================================

Model saved as mnist_ffnn.pth
